In [6]:
pip install torch torchvision


Defaulting to user installation because normal site-packages is not writeable
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 1.2 MB/s eta 0:00:0000:0100:020m
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 512.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 772.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 650.8 kB/s eta 0:00:0000:01m
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-

In [33]:
pip install git+https://github.com/FacePerceiver/facer.git@main

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/FacePerceiver/facer.git (to revision main) to /private/var/folders/_w/y0sjvgrs5jd1ml8yg11t_0f00000gn/T/pip-req-build-sbwmls87
  Running command git clone --filter=blob:none --quiet https://github.com/FacePerceiver/facer.git /private/var/folders/_w/y0sjvgrs5jd1ml8yg11t_0f00000gn/T/pip-req-build-sbwmls87
  Resolved https://github.com/FacePerceiver/facer.git to commit f0dd2d67def16445454b90a191cc763431795979
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install timm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import torchvision.transforms as transforms
HorizontalFlip_transformations = transforms.Compose([transforms.RandomHorizontalFlip(p=1.0)])  # Random horizontal flip
VerticalFlip_transformations = transforms.Compose([transforms.RandomVerticalFlip(p=1.0)])  # Use transforms.RandomVerticalFlip
Rotation30_transformations = transforms.Compose([transforms.RandomRotation(degrees=30)])
Rotation45_transformations = transforms.Compose([transforms.RandomRotation(degrees=45)])
Rotation60_transformations = transforms.Compose([transforms.RandomRotation(degrees=60)])
Rotation75_transformations = transforms.Compose([transforms.RandomRotation(degrees=75)])
Rotation90_transformations = transforms.Compose([transforms.RandomRotation(degrees=90)])
Rotation120_transformations = transforms.Compose([transforms.RandomRotation(degrees=120)])
Rotation150_transformations = transforms.Compose([transforms.RandomRotation(degrees=150)])
Rotation180_transformations = transforms.Compose([transforms.RandomRotation(degrees=180)])
Rotation210_transformations = transforms.Compose([transforms.RandomRotation(degrees=210)])
Rotation240_transformations = transforms.Compose([transforms.RandomRotation(degrees=240)])
Rotation270_transformations = transforms.Compose([transforms.RandomRotation(degrees=270)])
Crop_transformations = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0))
    # Changed to RandomResizedCrop, which supports the 'scale' argument
])
Affine_transformations = transforms.Compose([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=10)]) # Use transforms.RandomAffine
ColorJitter_transformations = transforms.Compose([transforms.ColorJitter(brightness=0.1, contrast=0.7, saturation=0.7, hue=0.3)]) # Use transforms.ColorJitter
GaussianBlur_transformations = transforms.Compose([transforms.GaussianBlur(kernel_size=5)])  # Use transforms.GaussianBlur

In [64]:
import numpy as np
import facer
def process_and_display_face_segmentation(image_path, model_name='retinaface/mobilenet',
                                          parser_name='farl/lapa/448', threshold=0.5,
                                          device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Process an image to perform face detection, segmentation, and crop up to the highest point of the eyebrows.

    Parameters:
    - image_path (str): Path to the image to process.
    - model_name (str): Name of the face detection model.
    - parser_name (str): Name of the face parsing model.
    - threshold (float): Threshold for binary mask creation.
    - device (str): Device for model execution ('cuda' or 'cpu').

    Returns:
    - np.ndarray: The cropped image array, or None if processing fails.
    """
    try:
        # Load the image and convert to BCHW format
        image = facer.hwc2bchw(facer.read_hwc(image_path)).to(device=device)

        # Load the face detector model
        face_detector = facer.face_detector(model_name, device=device)

        # Detect faces
        with torch.inference_mode():
            faces = face_detector(image)

        if faces is None or len(faces) == 0:
            print(f"No faces detected in {image_path}.")
            return None

        # Load the face parser model
        face_parser = facer.face_parser(parser_name, device=device)

        # Parse faces for segmentation
        with torch.inference_mode():
            faces = face_parser(image, faces)

        # Extract segmentation logits and probabilities
        seg_logits = faces['seg']['logits']
        seg_probs = seg_logits.softmax(dim=1)  # nfaces x nclasses x h x w

        # Define label names
        label_names = ['background', 'face', 'rb', 'lb', 're', 'le', 'nose', 'ulip', 'imouth', 'llip', 'hair']

        # Locate the highest point of the eyebrows
        if 'rb' in label_names and 'lb' in label_names:
            rb_index = label_names.index('rb')
            lb_index = label_names.index('lb')

            # Create binary masks for the eyebrows
            rb_mask = seg_probs[0, rb_index] > threshold
            lb_mask = seg_probs[0, lb_index] > threshold

            # Find the highest points (lowest y-coordinate in image space)
            rb_coords = np.column_stack(np.where(rb_mask.cpu().numpy()))
            lb_coords = np.column_stack(np.where(lb_mask.cpu().numpy()))

            highest_rb = rb_coords[np.argmin(rb_coords[:, 0])] if rb_coords.size > 0 else None
            highest_lb = lb_coords[np.argmin(lb_coords[:, 0])] if lb_coords.size > 0 else None

            # Determine the highest point overall
            highest_point = None
            if highest_rb is not None and highest_lb is not None:
                highest_point = highest_rb if highest_rb[0] < highest_lb[0] else highest_lb
            elif highest_rb is not None:
                highest_point = highest_rb
            elif highest_lb is not None:
                highest_point = highest_lb

            if highest_point is not None:
                # Crop the image up to the highest point of the eyebrows
                cropped_image = image.cpu().numpy()[0, :, highest_point[0]:, :].transpose(1, 2, 0)
                return cropped_image

        print(f"Eyebrows not found in {image_path}.")
        return None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None


In [42]:
def adjust_brightness_contrast(image, alpha=1.0, beta=0):
    """
    Adjust brightness and contrast of an image.
    Args:
        image (numpy array): Input image.
        alpha (float): Contrast control (1.0-3.0).
        beta (int): Brightness control (-100 to 100).
    Returns:
        numpy array: Adjusted image.
    """
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted
def simulate_lighting_conditions(image):
    lighting_conditions = []
    alphas = [0.5, 1.0, 1.5]  # Contrast values
    betas = [-50, 0, 50]      # Brightness values

    for alpha in alphas:
        for beta in betas:
            # Skip the condition where alpha=0.5 and beta=-50
            if alpha == 0.5 and beta == -50:
                continue
            if alpha == 1 and beta == 0:
                continue
            if alpha == 1.5 and beta == 50:
                continue
            adjusted = adjust_brightness_contrast(image, alpha=alpha, beta=beta)
            lighting_conditions.append((alpha, beta, adjusted))

    return lighting_conditions

In [43]:
from facer import face_parsing

In [70]:
import os
from PIL import Image
import cv2

def save_results_incrementally(results, output_dir, folder, base_name, current_index):
    """
    Saves augmented images from the results dictionary incrementally into the specified directory.

    Args:
        results (dict): Dictionary containing augmented images.
        output_dir (str): Path to the output directory.
        folder (str): Subfolder for the current dataset class.
        base_name (str): Base name for the images (e.g., "hiepnm", "lamnt").
        current_index (int): The current index for naming files.

    Returns:
        int: The next index to use for naming files in the same folder.
    """
    # Ensure the subdirectory exists
    subdir = os.path.join(output_dir, folder)
    os.makedirs(subdir, exist_ok=True)

    for category, images in results.items():
        for name, image in images.items():
            save_path = os.path.join(subdir, f"{base_name}_{current_index}.jpg")
            if isinstance(image, Image.Image):  # Check if it's a PIL image
                image.save(save_path)
            print(f"Saved: {save_path}")
            current_index += 1

    return current_index

def augment_and_simulate(image_path, output_dir, folder, current_index):
    """
    Simulates different lighting conditions and performs data augmentation,
    saving all results into the specified directory.

    Args:
        image_path: The path to the image file.
        output_dir: The directory where augmented images will be saved.
        folder: Subfolder for the current dataset class.
        current_index: The current index for naming files.

    Returns:
        int: The updated index after processing the image.
    """
    try:
        cropped_image = process_and_display_face_segmentation(image_path)
        if cropped_image is None:
            print(f"Skipping {image_path} as face processing failed.")
            return current_index
        # Load image using PIL for PyTorch augmentations
        pil_image = Image.fromarray(cropped_image)
        lighting_conditions = simulate_lighting_conditions(cropped_image)
        # Define augmentations
        transformations = {
            "Horizontal Flip": HorizontalFlip_transformations,
            # "Vertical Flip": VerticalFlip_transformations,
            "Rotation30": Rotation30_transformations,
            "Rotation45": Rotation45_transformations,
            "Rotation60": Rotation60_transformations,
            "Rotation75": Rotation75_transformations,
            "Rotation90": Rotation90_transformations,
            "Rotation120": Rotation120_transformations,
            "Rotation150": Rotation150_transformations,
            "Rotation180": Rotation180_transformations,
            "Rotation210": Rotation210_transformations,
            "Rotation240": Rotation240_transformations,
            "Rotation270": Rotation270_transformations,
            "Crop": Crop_transformations,
            "Affine": Affine_transformations,
        }

        # Simulate Lighting Conditions
        

        # Initialize results dictionary
        results = {}

        # Augment Original Image
        original_augmentations = {}
        for name, transform in transformations.items():
            augmented = transform(pil_image)
            original_augmentations[name] = augmented
        results["Original_Image"] = original_augmentations

        # Simulate Lighting for Original Image and Augment
        for idx, (alpha, beta, lighting_image) in enumerate(lighting_conditions):
            lighting_pil_image = Image.fromarray(lighting_image)
            augmented_lighting_images = {}
            for name, transform in transformations.items():
                augmented = transform(lighting_pil_image)
                augmented_lighting_images[name] = augmented
            results[f"Lighting_{idx + 1}_Alpha_{alpha}_Beta_{beta}"] = augmented_lighting_images

        # Extract base name without class index (e.g., "hiepnm" from "hiepnm_4.jpg")
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        base_name = "_".join(base_name.split("_")[:-1])  # Remove the last part after "_"

        # Save all augmented images
        return save_results_incrementally(results, output_dir, folder, base_name, current_index)

    except (FileNotFoundError, IOError):
        print(f"Error: Could not open or read image file at {image_path}")
        return current_index


# Main script
input_dir = "dataset_crop_test"
output_dir = "augmented_dataset_crop_test"

for folder in os.listdir(input_dir):
    subfolder_path = os.path.join(input_dir, folder)
    if os.path.isdir(subfolder_path):
        current_index = 1  # Start index for each folder
        for filename in sorted(os.listdir(subfolder_path)):  # Ensure sorted order
            image_path = os.path.join(subfolder_path, filename)
            if os.path.isfile(image_path):
                print(f"Processing image: {image_path}")
                current_index = augment_and_simulate(
                    image_path=image_path,
                    output_dir=output_dir,
                    folder=folder,
                    current_index=current_index
                )


Processing image: dataset_crop_test/0/hiepnm_1.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_1.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_2.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_3.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_4.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_5.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_6.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_7.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_8.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_9.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_10.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_11.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_12.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_13.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_14.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_15.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_16.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_17.jpg
Saved: augmented_dataset_crop_test/0/hiepnm_18.jpg
Saved: augmented_dataset_crop_test/0/hie